In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os,cv2

In [ ]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 10)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

In [2]:
directory="dataset/train/"
classses=os.listdir(directory)

In [3]:
import os
import cv2
image_size=224
data1=[]#empty list
for category in classses:
    path = os.path.join(directory, category)
    print(path)
    label = classses.index(category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        arr = cv2.imread(img_path)#Reading Images
        arr = cv2.cvtColor(arr, cv2.COLOR_BGR2RGB)
        new_arr = cv2.resize(arr, (image_size, image_size))#Resizing into same shape
        data1.append([new_arr, label])

dataset/train/COVID19
dataset/train/NORMAL
dataset/train/OTHERS
dataset/train/PNEUMONIA
dataset/train/TUBERCOLOSIS


In [4]:
import random
random.shuffle(data1)

In [5]:
len(data1)

7912

In [6]:
classses

['COVID19', 'NORMAL', 'OTHERS', 'PNEUMONIA', 'TUBERCOLOSIS']

In [7]:
x=[]#ndependent
y=[]#dependent
for features,labels in data1:
    x.append(features)
    y.append(labels)

In [8]:
x=np.array(x)
y=np.array(y)
#x=x/255 #feature scaling

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [10]:
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential ,Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense ,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
model = Sequential()
model.add(base_model)
model.add(Flatten(input_shape=base_model.output_shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model = Model(inputs=model.input, outputs=model.output)
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['accuracy'])

In [ ]:
progess = model.fit(x=X_train,y=y_train,epochs=5,validation_data= (X_test, y_test))

Epoch 1/5
198/198 [==============================] - 3523s 18s/step - loss: 0.2629 - accuracy: 0.9371 - val_loss: 0.0333 - val_accuracy: 0.9886
Epoch 2/5
 75/198 [==========>...................] - ETA: 36:01 - loss: 0.0474 - accuracy: 0.9862

In [ ]:
plt.plot(progess.history['accuracy'])
plt.plot(progess.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(progess.history['loss'])
plt.plot(progess.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)

In [ ]:
y_pred_new=[]
for i in y_pred:
    #print(i)
    #break
    y=np.argmax(i)
    #break
    #print(y)
    y_pred_new.append(y)
    #break

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_new, target_names=classses))

In [ ]:
from sklearn import metrics
cm=metrics.confusion_matrix(y_test,y_pred_new)
import pandas as pd
df_cm=pd.DataFrame(cm)
import seaborn as sns
plt.figure(figsize = (10,3))
sns.heatmap(df_cm, annot=True)

In [ ]:
model.save("VGGSKin.hp5")

In [ ]:
import os

In [ ]:
for i in os.listdir("testing/"):
    print(i)
    
    from tensorflow import keras
    model = keras.models.load_model('VGGSKin.hp5')
    from keras.preprocessing import image
    image_size=224
    #img = cv2.imread(path1+"//"+i)
    path="testing/"+i
    img = image.load_img(path, target_size=(image_size, image_size))
    x = image.img_to_array(img)
    print(type(x))
    img_4d=x.reshape(1,224,224,3)
    #img_4d=img_4d/255
    model = keras.models.load_model('VGGSKin.hp5')
    predictions = model.predict(img_4d)
    print(predictions[0])
    new_pred=np.argmax(predictions[0])
    print(new_pred)
#     ['COVID19', 'NORMAL', 'OTHERS', 'PNEUMONIA', 'TUBERCOLOSIS']
    dict1={0:'COVID19',1:'NORMAL',2:'OTHERS',3:'PNEUMONIA',4:'TUBERCOLOSIS'}
    list1=["COVID19","NORMAL","OTHERS","PNEUMONIA","TUBERCOLOSIS"]
    a=dict1[new_pred]
    
    print(str(a)+" Detected in Image")
    print("-----------------------------------------------")
    #break

In [ ]:
# pip install Pillow

In [ ]:
# from PIL import Image
# import os

# def resize_images(input_folder, output_folder, target_size):
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     for filename in os.listdir(input_folder):
#         if filename.endswith(('.jpg', '.png', '.jpeg')):  # Add more supported extensions if needed
#             image_path = os.path.join(input_folder, filename)
#             img = Image.open(image_path)
            
#             # Resize the image while maintaining the aspect ratio
#             img.thumbnail(target_size)

#             # Add padding to make the image exactly 512x512
#             background = Image.new('RGB', target_size, (255, 255, 255))
#             background.paste(img, ((target_size[0] - img.size[0]) // 2, (target_size[1] - img.size[1]) // 2))

#             output_path = os.path.join(output_folder, filename)
#             background.save(output_path)

# input_folder = 'dataset/train/OTHERS'  # Replace with the path to your input folder
# output_folder = 'dataset/train/newfolder'  # Replace with the path to your output folder
# target_size = (512, 512)

# resize_images(input_folder, output_folder, target_size)
